In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import  RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold
import eli5
from eli5.sklearn import PermutationImportance

In [3]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/dw_matrix/matrix_two/dw_matrix_car'

/content/drive/My Drive/Colab Notebooks/matrix/dw_matrix/matrix_two/dw_matrix_car


In [4]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [5]:
df = df[df['price_currency'] == 'PLN']
df.shape

(106290, 155)

In [0]:
SUFFIX_CAT = '__cat'

for feat in df.columns:
  if SUFFIX_CAT in feat: continue
  if isinstance(df[feat][0], list): continue
  df[feat + SUFFIX_CAT] = df[feat].factorize()[0]

In [8]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  model = model
  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

##Decision Tree Regressor

In [17]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19566.588937368324, 90.6181486516617)

##Random Forest

In [18]:
model = RandomForestRegressor(n_estimators=50, max_depth=5, random_state=0)
run_model(model, cat_feats)

(-18734.2072708522, 109.87074106274046)

##XGBoost

In [22]:
xgb_params = {
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed': 0
}
model = xgb.XGBRegressor(**xgb_params)
run_model(model, cat_feats)

[07:10:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:10:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:11:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13039.290196724838, 109.36715375706265)

##Feature importance - eli5

In [23]:
m = xgb.XGBRegressor(**xgb_params)
m.fit(X, y)

imp = PermutationImportance(m, random_state=0).fit(X, y)
eli5.show_weights(imp, feature_names=cat_feats)

[07:11:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1209 ± 0.0019,param_napęd__cat
0.1175 ± 0.0030,param_rok-produkcji__cat
0.1113 ± 0.0013,param_stan__cat
0.0625 ± 0.0019,param_skrzynia-biegów__cat
0.0527 ± 0.0016,param_faktura-vat__cat
0.0461 ± 0.0015,param_moc__cat
0.0275 ± 0.0008,param_marka-pojazdu__cat
0.0230 ± 0.0004,param_typ__cat
0.0227 ± 0.0007,feature_kamera-cofania__cat
0.0191 ± 0.0007,param_pojemność-skokowa__cat


In [0]:
feats = ['param_napęd__cat',
'param_rok-produkcji',
'param_stan__cat',
'param_skrzynia-biegów__cat',
'param_faktura-vat__cat',
'param_moc',
'param_marka-pojazdu__cat',
'param_typ__cat',
'feature_kamera-cofania__cat',
'param_pojemność-skokowa',
'seller_name__cat',
'param_kod-silnika__cat',
'param_model-pojazdu__cat',
'feature_wspomaganie-kierownicy__cat',
'param_wersja__cat',
'feature_czujniki-parkowania-przednie__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_regulowane-zawieszenie__cat',
'feature_system-start-stop__cat',
'feature_światła-led__cat']



##XGBoost with fewer features

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if x is None else int(x))

In [0]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if x is None else int(x.split(' ')[0]))

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if x is None else int(x.split('cm')[0].replace(' ', '')))

In [43]:
model = xgb.XGBRegressor(**xgb_params)
run_model(model, feats)

[07:50:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:50:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:50:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9449.513980284812, 81.47168211987172)